In [47]:
import pandas as pd
import os
import re
import numpy as np
import datetime
from pathlib import Path

In [48]:
def fasta_get_by_name(fasta, ids, delimit=" ", column=0):
	seq = {}
	key = ""
	array = []
	search = {}
	# keep fasta in dict
	with open(fasta,'r') as f:        
		for line in f: 
			line = line.rstrip()
			if '>' in line:
				key = line.replace('>', '')
				seq[key] = ""
				continue
			seq[key] = seq[key] + line
	# create pandas dataframe from dict
	for k in seq.keys():
		id_ = k.split(delimit)[column]
		array.append([k, id_, seq[k]])
	df = pd.DataFrame(array)
	df.columns = ['header', 'id', 'seq']
	# seq selecciotion
	df = df.loc[df['id'].isin(ids)]
	for i in df.index:
		search[df.loc[i]['header']] = df.loc[i]['seq']
	return search

In [49]:
### paths generales
base = Path('/mnt/NAS/projects/virus_SSPA/COVID19_SSPA/')
nextstrain_data = base / 'nextstrain_all/data'
wave1_path = base / "other_data/1wave_data/"
ssp = base / 'secuenciacion_salud_publica/'
qc_path = ssp / 'analysis_MN908947/all_run_QC_results/'
metadata_path = ssp / 'metadata/'

In [50]:
### QC muetras
### Genera el dataframe samples original
### fichero con los path de cada batch
bp_file = ssp / 'sarscov2_sequencing_analysis_info.csv'
bp = pd.read_csv(bp_file, low_memory=False)

### concateno las tablas de samples all qc
qc_array = []
for b, i, v in bp[['run_id', 'path_QC', 'viralrecon_version']].values:
    qc = pd.read_csv(i, dtype={'laboratoryID':str, 'batch':str})
    
    # estas columnas solo estan en algunas de los primeros qc
    drop_col = ['qc_nextclade', 'totalGaps_nextclade', 'totalMutations_nextclade']
    qc.drop(columns=drop_col, inplace=True, errors='ignore')
    
    if v != 'iontorrent':
        qc['tecnología'] = 'illumina'
    else: qc['tecnología'] = v
        
    if b == 'AND_2_husc_010221':
        # elimina el 1442 al inicio que no debía de estar ahí ya que viene de un id de roche
        qc['laboratoryID'] = qc['laboratoryID'].apply(lambda x: re.sub(r'^1442', '', x))
    qc_array.append(qc)

samples = pd.concat(qc_array)
samples.reset_index(inplace=True, drop=True)

### Elimino los prefijos que se utilizan en HUSC jusnto a los ID de laboratorio
samples.loc[samples['laboratoryID'].str.len() == 12, 'laboratoryID'] = samples['laboratoryID'].str.replace(r'^1442', '', regex=True)
samples.loc[samples['laboratoryID'].str.len() == 12, 'laboratoryID'] = samples['laboratoryID'].str.replace(r'^1012', '', regex=True)
samples.loc[samples['laboratoryID'].str.len() == 10, 'laboratoryID'] = samples['laboratoryID'].str.replace(r'^47', '', regex=True)

In [51]:
### mapping de identificadores
### concateno las tablas de mapping ids
lab_array = []

for b in bp['run_id'].values:
    if 'huvr' in b:
        lab = pd.read_csv(f'{metadata_path}/huvr/{b}/{b}_sampleIDlabo_mapping_andID.csv', header=None, dtype={2:str})
    elif 'husc' in b:
        lab = pd.read_csv(f'{metadata_path}/husc/{b}/{b}_sampleIDlabo_mapping_andID.csv', header=None, dtype={2:str})
        if b == 'AND_2_husc_010221':
            # elimina el 1442 al inicio que no debía de estar ahí ya que viene de un id de roche
            lab[2] = lab[2].apply(lambda x: re.sub(r'^1442', '', x)) 
    elif 'huvn' in b:
        lab = pd.read_csv(f'{metadata_path}/huvn/{b}/{b}_sampleIDlabo_mapping_andID.csv', header=None, dtype={2:str})
    elif 'hrum' in b:
        lab = pd.read_csv(f'{metadata_path}/hrum/{b}/{b}_sampleIDlabo_mapping_andID.csv', header=None, dtype={2:str})
    lab_array.append(lab)
    
mapping = pd.concat(lab_array)
mapping.columns = ['sample', 'and_id', 'num_lab']
mapping.reset_index(inplace=True, drop=True)
# remove complete duplicated
mapping = mapping.loc[~mapping.duplicated(keep='first')].copy()
mapping.reset_index(inplace=True, drop=True)
# remove some prefix
mapping.loc[mapping['num_lab'].str.len() == 12, 'num_lab'] = mapping['num_lab'].str.replace(r'^1442', '', regex=True)
mapping.loc[mapping['num_lab'].str.len() == 12, 'num_lab'] = mapping['num_lab'].str.replace(r'^1012', '', regex=True)
mapping.loc[mapping['num_lab'].str.len() == 10, 'num_lab'] = mapping['num_lab'].str.replace(r'^47', '', regex=True)

In [52]:
### tablas linajes de pangolin
### concateno las tablas de linajes de pangolin
lin_array = []
for b, i in bp[['run_id', 'analysis_path']].values:
    file = f'{i}/pangolin/{b}_pangolin.csv'
    lin = pd.read_csv(file, sep=',')
    lin_array.append(lin)
    
### añado los linajes de la primera ola y proyecto nacional
wave1_lineage = wave1_path / 'pangolin/andalusia_seq_pangolin.csv'
wave1_lineage_df = pd.read_csv(wave1_lineage, sep=',')
lin_array.append(wave1_lineage_df)

lineage = pd.concat(lin_array)

In [53]:
### concateno las tablas de nextclade
nextclade_array = []
for a_path, run in bp[['analysis_path', 'run_id']].values:
    file = Path(a_path) / f'nextclade/{run}.csv'
    nc = pd.read_csv(file, sep=';')
    nextclade_array.append(nc)
    
### añado los clados de la primera ola
wave1_nextclade = wave1_path / 'nextclade/andalusia_seq_nextclade.csv'
wave1_nextclade_df = pd.read_csv(wave1_nextclade, sep=';')
nextclade_array.append(wave1_nextclade_df)
    
nextclade = pd.concat(nextclade_array)
nextclade = nextclade[['seqName', 'clade', 'clade_display', 'clade_who', 'Nextclade_pango']]

In [54]:
nextclade.loc[nextclade['seqName'] == "55017021"]

,seqName,clade,clade_display,clade_who,Nextclade_pango
311,55017021,20A,20A,NaN,B.1


In [55]:
### Tablas de metadatos
### Concateno los fichero de metadatos
dtype = {
    'IDlaboratorio':str, 
    'Código Postal':str, 
    'Edad (años)':"Int64",
    'Nº Contactos identificados':str,
    'Identificador':"Int64",
    'Tipo':str
}

remove_cols = [
    'Evolución',
    'Fecha Alta (dd/mm/aaaa)',
    'Fecha de defunción',
    'Fue Hospitalizado',
    'Hospital de ingreso',
    'Identificador',
    'Identificador Brote asociado',
    'Tipo',
    'Ámbito',
    'Clasificación reinfección',
    'Otros cuadros respiratorios graves',
    'Asintomático (sin antecedentes de síntomas)',
    'Categoría profesional',
    'Contacto estrecho',
    'Diabetes',
    'Embarazo',
    'Enfermedad cardiovascular',
    'Enfermedad pulmonar crónica',
    'Estancia en UCI',
    'Factores de Riesgo y Enfermedad de Base COVID-19',
    'Fallo renal agudo',
    'Fecha alta UCI',
    'Fecha ingreso UCI',
    'Hipertensión arterial (HTA)',
    'Inmunosupresión',
    'Institucionalizados (revisar ámbito)',
    'Neumonía',
    'Otros factores',
    'Otros factores, especificar',
    'Síndrome Distress respiratorio',
    'Ventilación mecánica',
    'Cáncer',
    'Estudio Contactos',
    'Fecha primera vacuna',
    'Fecha segunda vacuna',
    'Nº Contactos confirmados como caso',
    'Nº Contactos identificados',
    'Nombre primera vacuna',
    'Nombre segunda vacuna',
    'FERE1',
    'RE1',
    'FERE2',
    'RE2',
    'FERE3',
    'RE3',
    'DATABASE RLT2_NUHSA'
]

# Original mets
met1 = pd.read_csv(f'{metadata_path}/huvr/AND_1_huvr_050121/AND_1_huvr_050121_sp.csv', sep='|', dtype=dtype)
met2 = pd.read_csv(f'{metadata_path}/husc/AND_2_husc_010221/AND_2_husc_010221_sp.csv', sep='|', dtype=dtype)
met3 = pd.read_csv(f'{metadata_path}/huvr/AND_3_huvr_020221/AND_3_huvr_020221_sp.csv', sep='|', dtype=dtype)
met4 = pd.read_csv(f'{metadata_path}/husc/AND_4_husc_200221/AND_4_husc_200221_sp.csv', sep='|', dtype=dtype)
met5 = pd.read_csv(f'{metadata_path}/huvr/AND_5_huvr_240221/AND_5_huvr_240221_sp.csv', sep='|', dtype=dtype)
met6 = pd.read_csv(f'{metadata_path}/huvr/AND_6_huvr_270221/AND_6_huvr_270221_sp.csv', sep='|', dtype=dtype)
met7 = pd.read_csv(f'{metadata_path}/huvr/AND_7_huvr_080321/AND_7_huvr_080321_sp.csv', sep=',', dtype=dtype)
ori_met = pd.concat([met1, met2, met3, met4, met5, met6, met7])
ori_met['ori'] = 'ori_met'

### a partir de aquí empezamos con los latest
met_huvr = pd.read_csv(f'{metadata_path}/huvr/huvr_acumulativo_latest_sp.csv', sep='|', low_memory=False, dtype=dtype)
met_huvr['ori'] = 'met_huvr'
met_husc = pd.read_csv(f'{metadata_path}/husc/husc_acumulativo_latest_sp.csv', sep='|', low_memory=False, dtype=dtype)
met_husc['ori'] = 'met_husc'
met_huvn = pd.read_csv(f'{metadata_path}/huvn/huvn_acumulativo_latest_sp.csv', sep='|', low_memory=False, dtype=dtype)
met_huvn['ori'] = 'met_huvn'
met_hrum = pd.read_csv(f'{metadata_path}/hrum/hrum_acumulativo_latest.csv', sep='|', low_memory=False, dtype=dtype)
met_hrum['ori'] = 'met_hrum'
latest_met = pd.concat([met_huvr, met_husc, met_huvn, met_hrum])

ori_met = ori_met.loc[(~ori_met['IDlaboratorio'].isin(latest_met['IDlaboratorio'])) & (~ori_met['NUHSA'].isin(latest_met['NUHSA']))]

### Este no lo usamos
# met_hurm = pd.read_csv(f'{metadata_path}/hrum/hrum_acumulativo_latest.csv', sep=';', dtype={'IDlaboratorio':str, 'Código Postal':str})

mets = [latest_met, ori_met]
metadata = pd.concat(mets)
metadata_original = metadata.copy()
metadata.drop(columns=remove_cols, inplace=True)
metadata.drop_duplicates(inplace=True, keep='first')
metadata.reset_index(inplace=True, drop=False)

### guardo en una tabla los ids de laboratorio duplicados
num_lab_duplicated = metadata.loc[metadata.duplicated(['IDlaboratorio'], keep=False)].sort_values(by='IDlaboratorio').shape[0]
if num_lab_duplicated > 0:
    print(f'Warning!! {num_lab_duplicated} números de laboratorio duplicados en el dataset metadata')
    print(metadata.loc[metadata.duplicated(['IDlaboratorio'], keep=False), ['IDlaboratorio', 'NUHSA']].sort_values(by='IDlaboratorio').head())
metadata.loc[metadata.duplicated('IDlaboratorio', keep=False)].sort_values(by='IDlaboratorio').to_excel('metadata_laboratory_id_duplicated.xlsx', index=False)

### elimino los id de laboratorio duplicados quedándome con el primero
metadata.drop_duplicates('IDlaboratorio', inplace=True, keep='first')

### cambios en los metadatos ad-hoc para que almería empiece por 0
metadata.loc[(metadata['Código Postal'].str.len() == 4) & (metadata['Código Postal'].str.match('^4')), 'Código Postal'] = '0' + metadata['Código Postal']

#### le quito los espacios en blanco a los nobres de municipio de la tabla metadata
metadata['Municipio'] = metadata['Municipio'].str.strip()

Warning!! 1148 números de laboratorio duplicados en el dataset metadata
      IDlaboratorio         NUHSA
36708      10130349  AN0285479283
36707      10130349  AN0285479283
20510      10139710  AN0067854834
43550      10139710  AN0067854834
5300      102057073  an0472741625


In [56]:
metadata[['Código Postal', 'Municipio']]

,Código Postal,Municipio
0,NaN,NaN
1,41091,Sevilla
2,41091,Sevilla
3,41040,Espartinas
4,41091,Sevilla
...,...,...
44904,11405,Jerez de la Frontera
44905,11405,Jerez de la Frontera
44906,11520,Rota
44907,11406,Jerez de la Frontera


In [57]:
### Unifico las tablas anteriores
### Mergeo el dataframe samples con lineages, nextclade y metadata
### mapping no se para que sirve y lo voy a quitar

print(f'total samples: {samples.shape[0]}')

### merge with lineage
samples = samples.merge(lineage, how='left', left_on='sample', right_on='taxon')
print(f'samples merged witn lineage: {samples.shape[0]}')

### merge with nextclade
samples = samples.merge(nextclade, how='left', left_on='sample', right_on='seqName')
print(f'samples merged wit nextclade: {samples.shape[0]}')


### merge with mapping
#samples = samples.merge(mapping, how='left', on='sample')
#print(f'samples merged witn mapping: {samples.shape[0]}')

#num_lab_error = len(samples.loc[samples['laboratoryID'] != samples['num_lab']][['laboratoryID', 'num_lab']].dropna())
#if num_lab_error != 0:
#    print(f'Warning!! {num_lab_error} Números de laboratorio diferentes en el dataset samples')
#    print(samples.loc[samples['laboratoryID'] != samples['num_lab'], ['laboratoryID', 'num_lab']].head(50))
    
#samples.loc[samples['laboratoryID'].isnull(), 'laboratoryID'] = samples['num_lab']

### merge with metadata
samples = samples.merge(metadata, how='left', left_on='laboratoryID', right_on='IDlaboratorio')
print(f'samples merged witn metadata: {samples.shape[0]}')

### Elimino las líneas identicas en el dataframe samples
samples.drop_duplicates(inplace=True, keep='first')
print(f'samples without duplicated: {samples.shape[0]}')

### Compruebo números de laboratorio duplicados en el dataset samples
samples_duplicated_labID = samples.loc[samples['laboratoryID'].duplicated(keep=False)].sort_values('laboratoryID').shape[0]
if samples_duplicated_labID > 0 :
    samples.loc[samples['laboratoryID'].duplicated(keep=False)].sort_values('laboratoryID').to_excel('samples_laboratory_id_duplicated.xlsx', index=False)
    print(f'Warning!! {samples_duplicated_labID} Números de laboratorio duplicados en el dataset samples')

total samples: 43385
samples merged witn lineage: 43385
samples merged wit nextclade: 43385
samples merged witn metadata: 43385
samples without duplicated: 43385
Warning!! 653 Números de laboratorio duplicados en el dataset samples


In [58]:
samples

,sample,fastq_sample,laboratoryID,seqPlatform,bioanalyzer_profile,num_input_reads,num_trimmed_reads,percentage_nonhost_reads,percentage_mapping_viral_genome,median_depth,...,Edad (años),Fecha del caso,Fecha Inicio Síntomas (dd/mm/aaaa),País del caso,Sexo,Centro de salud,Código Postal,Municipio,Fecha_diag,ori
0,AND00001,HUVR-2UK-40085344,HUVR-2UK-40085344,Illumina,-,1440704,1200502,99.851895,99.79,4672.0,...,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AND00002,HUVR-7UK-50072114,HUVR-7UK-50072114,Illumina,-,1297010,1079260,99.752423,99.71,4462.0,...,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AND00003,HUVR-4UK-40085183,HUVR-4UK-40085183,Illumina,-,1427538,1188870,99.896036,99.89,4613.0,...,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AND00004,HUVR-6UK-50072094,HUVR-6UK-50072094,Illumina,-,1351068,1131586,99.945740,99.94,4750.0,...,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AND00005,HUVR-1UK-201314254,HUVR-1UK-201314254,Illumina,-,1458662,1229806,99.381041,99.37,4807.0,...,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43380,AND43614,41231737,41231737,Illumina,-,336848,244960,99.997551,99.91,895.0,...,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,met_huvn
43381,AND43615,41231741,41231741,Illumina,-,376174,269114,99.820151,99.7,928.0,...,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,met_huvn
43382,AND43616,41231749,41231749,Illumina,-,308776,224528,99.944773,99.81,823.0,...,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,met_huvn
43383,AND43617,41231750,41231750,Illumina,-,307906,218482,99.972538,99.85,772.0,...,<NA>,NaN,NaN,NaN,NaN,Santa Rosa,14021,Córdoba,27/03/2022,met_huvn


In [59]:
### Borrado de muestras incorrectas
### Elimino muestras por errores en los datos que nos pasan
samples = samples.loc[samples['sample'] != 'AND37139']

In [60]:
print(samples.shape)
print(mapping.shape)
print(metadata.shape)
print(lineage.shape)

(43384, 66)
(43362, 3)
(44156, 17)
(43885, 16)


In [61]:
print(samples.shape)
print(mapping.shape)
print(metadata.shape)
print(lineage.shape)

(43384, 66)
(43362, 3)
(44156, 17)
(43885, 16)


In [62]:
sam_ids = set(samples['sample'].values)
lin_ids = set(lineage['taxon'].values)

len(sam_ids - lin_ids)

517

In [63]:
##################################################################################################################
# Modificaciones posteriores a la tabla samples mergeada con todo

#########################################
# asigno la fuente
samples['origen'] = 'SAS'

#########################################
# Asigno las provincias por código postal
samples.loc[samples['Código Postal'].isnull(), 'Código Postal'] = '0'
samples.loc[samples['Código Postal'].str.startswith('41'), 'provincia'] = 'Sevilla'
samples.loc[samples['Código Postal'].str.startswith('14'), 'provincia'] = 'Córdoba'
samples.loc[samples['Código Postal'].str.startswith('21'), 'provincia'] = 'Huelva'
samples.loc[samples['Código Postal'].str.startswith('23'), 'provincia'] = 'Jaén'
samples.loc[samples['Código Postal'].str.startswith('18'), 'provincia'] = 'Granada'
samples.loc[samples['Código Postal'].str.startswith('29'), 'provincia'] = 'Málaga'
samples.loc[samples['Código Postal'].str.startswith('04'), 'provincia'] = 'Almería'
samples.loc[samples['Código Postal'].str.startswith('11'), 'provincia'] = 'Cádiz'



############################################
# conversion de nombres de localidades al formato de la tabla de latitud y longitud
samples['Municipio'] = samples['Municipio'].str.strip()

location_names = './data/location_names_dict.tsv'
with open(location_names,'r') as f:
    for line in f: 
        line = line.strip()
        loc1, loc2 = line.split('\t')
        samples.loc[samples['Municipio'] == loc1, 'Municipio'] = loc2
        
lat_long = pd.read_csv('./config/lat_long.tsv', sep='\t', names=['location', 'name', 'lat', 'long'])
print('Municipios sin localización')
print(samples.loc[(~samples['Municipio'].isin(lat_long['name'])) & (samples['Municipio'].notnull()), 'Municipio'])
print('########')

#######################################################
# Asginación de provincias a partir de la localidad (para cuando no hay codigo postal)
loc_province = f'./data/location_province_dict.tsv'
with open(loc_province,'r') as f:
    for line in f: 
        line = line.rstrip()
        loc, province = line.split('\t')
        samples.loc[samples['Municipio'] == loc, 'provincia'] = province
        
print('Municipios sin provincia\n', samples.loc[(samples['provincia'].isnull()) & (samples['Municipio'].notnull()), 'Municipio'].value_counts())

########################################################################
# genera los hospitales y los hospitales de referencia para las muestras

# Hospitales de referencia
samples.loc[samples['batch'].str.contains('huvr'), 'hospital de referencia'] = 'HUVR'
samples.loc[samples['batch'].str.contains('husc'), 'hospital de referencia'] = 'HUSC'

# Hospitales recolector
samples.loc[samples['provincia'] == 'Sevilla', 'hospital'] = 'Hospital Universitario Virgen del Rocío'
samples.loc[samples['provincia'] == 'Córdoba', 'hospital'] = 'Hospital Universitario Reina Sofía'
samples.loc[samples['provincia'] == 'Huelva', 'hospital'] = 'Hospital Universitario Juan Ramón Jiménez'
samples.loc[samples['provincia'] == 'Jaén', 'hospital'] = 'Hospital Universitario de Jaén'
samples.loc[samples['provincia'] == 'Granada', 'hospital'] = 'Hospital Universitario San Cecilio'
samples.loc[samples['provincia'] == 'Málaga', 'hospital'] = 'Hospital Universitario Virgen de la Victoria'
samples.loc[samples['provincia'] == 'Almería', 'hospital'] = 'Hospital Universitario Torrecárdenas'
samples.loc[samples['provincia'] == 'Cádiz', 'hospital'] = 'Hospital Universitario Puerta del Mar'

samples.loc[samples['batch'].str.contains('huvn'), 'hospital'] = 'Hospital Universitario Vigen de las Nieves'
samples.loc[samples['batch'].str.contains('hurm'), 'hospital'] = 'Hospital Universitario Regional de Málaga'

########################################################
# Elimina los duplicados.Filas identicas
samples.drop_duplicates(keep='first', inplace=True)

########################################################
# Completo los metadatos de las secuencias que no tienen asignada plataforma de secuenciación
samples.loc[samples['seqPlatform'].isnull(), 'seqPlatform'] = 'Illumina'

########################################################
# Por peticion de Lepe las muestras del HUVR pasan a tener como centro de secuenciación "microbiología HU Virgen del Rocio"
samples.loc[samples['batch'].str.contains('huvr'), 'centro_secuenciacion'] = "microbiología HU Virgen del Rocio"

samples.loc[samples.duplicated(keep=False)].shape

/tmp/ipykernel_50290/1163936512.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  samples['origen'] = 'SAS'
/tmp/ipykernel_50290/1163936512.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  samples.loc[samples['Código Postal'].str.startswith('41'), 'provincia'] = 'Sevilla'
/tmp/ipykernel_50290/1163936512.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:/

Municipios sin localización
11735    Fernán Núñez
11736    Fernán Núñez
11737    Fernán Núñez
11884     Cúllar-Vega
11894    Huétor-Tájar
11897    Huétor-Tájar
11927    Huétor-Tájar
11928    Huétor-Tájar
11939        Santa Fé
12153    Fernán Núñez
12155    Fernán Núñez
12156    Fernán Núñez
12157    Fernán Núñez
12158    Fernán Núñez
12159    Fernán Núñez
12160    Fernán Núñez
12161    Fernán Núñez
12285    Huétor-Tájar
12568    Fernán Núñez
12575    Fernán Núñez
12576    Fernán Núñez
12577    Fernán Núñez
12578    Fernán Núñez
12579    Fernán Núñez
12880    Fernán Núñez
12881    Fernán Núñez
19483       Taha (La)
26760     Cúllar-Vega
26769     Cúllar-Vega
26813     Cúllar-Vega
26850        Santa Fé
26851        Santa Fé
26853        Santa Fé
26854        Santa Fé
26855        Santa Fé
26856        Santa Fé
27531    Fernán Núñez
27707     Cúllar-Vega
27713        Santa Fé
27715        Santa Fé
27993     Cúllar-Vega
28110    Fernán Núñez
28124    Fernán Núñez
28275    Fernán Núñez
2829

/tmp/ipykernel_50290/1163936512.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  samples.loc[samples['batch'].str.contains('huvr'), 'hospital de referencia'] = 'HUVR'
/tmp/ipykernel_50290/1163936512.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  samples.loc[samples['provincia'] == 'Sevilla', 'hospital'] = 'Hospital Universitario Virgen del Rocío'
/tmp/ipykernel_50290/1163936512.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the

(0, 70)

In [64]:
temp = samples[['Municipio', 'provincia']]

temp = temp.loc[temp['Municipio'].notnull()]
temp = temp.loc[temp['provincia'].notnull()]
temp = temp.drop_duplicates()
temp.to_csv('test_location_province.tsv', sep='\t', index=False)

In [65]:
#########################################
# Unifico las fechas prodecedentes de los datos de salud pública
samples['fecha_nextstrain'] = samples['Fecha Inicio Síntomas (dd/mm/aaaa)']
samples.loc[samples['fecha_nextstrain'].isnull(), 'fecha_nextstrain'] = samples['Fecha del caso']
samples.loc[samples['fecha_nextstrain'].isnull(), 'fecha_nextstrain'] = samples['Fecha_diag']
# Transforma las fecha de string a datetime
samples['fecha_nextstrain'] = pd.to_datetime(samples['fecha_nextstrain'], format='%d/%m/%Y')

########################################################
# Actualizo las fechas según los datos de Microbiología
############ Andalucía occidental #####################
print('#########################################')
print('Andalucia Occidental (Microbiología HUVR)')
print('#########################################')
date_occident_path = f'{metadata_path}/huvr/MUESTRAS_SECUENCIACION_SEMANALES_acumulada_latest.csv'
date_occi = pd.read_csv(date_occident_path, sep=';')

### Comprueba que todas las fechas están en formato adecuado
date_occi_errors = date_occi[['Identificacion', 'NUHSA', 'Fecha']].loc[pd.to_datetime(date_occi['Fecha'], format='%d/%m/%Y', errors='coerce').isnull()]
if date_occi_errors.shape[0] > 0:
    print('>>> Fechas formato dd/mm/yyyy: ')
    print(date_occi[['Identificacion', 'NUHSA', 'Fecha']].loc[pd.to_datetime(date_occi['Fecha'], format='%d/%m/%Y', errors='coerce').isnull()])
    # Me quedo con los errores para mandarlos a micro
    date_occi['Fecha'] = pd.to_datetime(date_occi['Fecha'], format='%d/%m/%Y', errors='coerce')
else:
    print('>>> Fechas formato OK')
    date_occi['Fecha'] = pd.to_datetime(date_occi['Fecha'], format='%d/%m/%Y')

### Comprueba que no hay fechas del futuro
today = datetime.datetime.now()
if date_occi.loc[pd.to_datetime(date_occi['Fecha']) > today].shape[0]:
    print('>>> Fechas futuro: ')
    print(date_occi[['Identificacion', 'NUHSA', 'Fecha']].loc[date_occi['Fecha'] > today])
    # Concateno con los errores de formato
    if date_occi_errors.shape[0] == 0:
        date_occi_errors
    else:
        date_occi_errors = pd.concat([date_occi_errors, date_occi[['ID', 'NUHSA', 'Fecha']].loc[date_occi['Fecha'] > today]])
    date_occi['Fecha'] = date_occi['Fecha'].loc[date_occi['Fecha'] <= today]
else:
    print('>>> Fechas futuro OK')

### Comprueba que no hay fechas anteriores al 1 enero de 2020
if date_occi.loc[date_occi['Fecha'] < '2020-01-01'].shape[0]:
    print('>>> Fechas antiguas: ')
    print(date_occi[['Identificacion', 'NUHSA', 'Fecha']].loc[date_occi['Fecha'] < '2020-01-01'])
    # Concateno con el resto de errores de fecha
    date_occi_errors = pd.concat([date_occi_errors, date_occi[['Identificacion', 'NUHSA', 'Fecha']].loc[date_occi['Fecha'] < '2020-01-01']])
    date_occi['Fecha'] = date_occi['Fecha'].loc[date_occi['Fecha'] >= '2020-01-01']
else:
    print('>>> Fechas antiguas OK')

# Guardo los errores en un fichero para pasarselo a Javier
date_occi_errors.to_csv('date_occi_error.tsv', sep='\t', index=False)

# elimino las fechas duplicadas con mismo id
date_occi.drop(date_occi.loc[date_occi[['Fecha', 'Identificacion']].duplicated(keep='first')].index, inplace=True)
date_occi.drop(date_occi.loc[date_occi[['NUHSA', 'Identificacion']].duplicated(keep='first')].index, inplace=True)

# unifico las tablas de samples y microbiología occidental
samples = samples.merge(date_occi[['Fecha', 'Identificacion']], how='left', left_on='laboratoryID', right_on='Identificacion')

# renombro la columna a fecha mircro y la paso a datetime
samples['Fecha_micro'] = samples['Fecha']
samples.rename(columns={'Fecha':'Fecha_micro_huvr'}, inplace=True)
samples['Fecha_micro'] = pd.to_datetime(samples['Fecha_micro'], format='%m/%d/%Y')

############### Modificació para añadir las muestras de la carcel de Córdoba ##################################
carcel = date_occi.loc[date_occi['Justificacion clinica'] == 'Brote prisión']
carcel = carcel.loc[~carcel['NUHSA'].str.startswith('AN'), ['Identificacion']]
carcel_ids = list(carcel['Identificacion'].values)
samples.loc[samples['laboratoryID'].isin(carcel_ids), 'Municipio'] = 'Córdoba'
samples.loc[samples['laboratoryID'].isin(carcel_ids), 'provincia'] = 'Córdoba'

print(samples.loc[samples[['sample', 'laboratoryID']].duplicated(keep=False)][['sample', 'laboratoryID', 'fecha_nextstrain']].values)

/tmp/ipykernel_50290/1130609688.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  samples['fecha_nextstrain'] = samples['Fecha Inicio Síntomas (dd/mm/aaaa)']
/tmp/ipykernel_50290/1130609688.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  samples['fecha_nextstrain'] = pd.to_datetime(samples['fecha_nextstrain'], format='%d/%m/%Y')


#########################################
Andalucia Occidental (Microbiología HUVR)
#########################################
>>> Fechas formato OK
>>> Fechas futuro OK
>>> Fechas antiguas: 
      Identificacion         NUHSA      Fecha
21371       14392132  AN0163330823 1967-09-05
[['AND26568' '1512915' Timestamp('2022-06-07 00:00:00')]
 ['AND26568' '1512915' Timestamp('2022-06-07 00:00:00')]]


/tmp/ipykernel_50290/1130609688.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  date_occi_errors = pd.concat([date_occi_errors, date_occi[['Identificacion', 'NUHSA', 'Fecha']].loc[date_occi['Fecha'] < '2020-01-01']])


In [66]:
########################################################
# Actualizo las fechas según los datos de Microbiología
############ Andalucía oriental ########################
print('#########################################')
print('Andalucia Oriental (Microbiología HUSC) #')
print('#########################################')

date_orient_path = f'{ssp}/metadata/husc/Actualizacion_Salud_Publica_latest.csv'
#date_orient_path = f'{ssp}/metadata/husc/Actualizacion_Salud_Publica_20240620_all.csv'
#date_ori = pd.read_csv(date_orient_path, sep=';')
date_ori = pd.read_csv(date_orient_path, sep=';', usecols=['Identificador', 'NUHSA', 'Fecha de la pueba'])

# Cambio el nombre de la columna identificador
date_ori.rename(columns={'Identificador':'ori_temp_id'}, inplace=True)
# Guardo las muestras sin fecha en un dataframe para reportar errores
date_ori_errors = date_ori[['ori_temp_id', 'NUHSA', 'Fecha de la pueba']].loc[date_ori['Fecha de la pueba'].isnull()]
# Elinimo las muestras sin fecha
date_ori = date_ori.loc[date_ori['Fecha de la pueba'].notnull()]
# Elimino las fechas sin los divisores correctos
date_ori = date_ori.loc[date_ori['Fecha de la pueba'].str.contains('/')]

### Elimino los prefijos que se utilizan en HUSC jusnto a los ID de laboratorio
date_ori.loc[date_ori['ori_temp_id'].str.len() == 12, 'ori_temp_id'] = date_ori['ori_temp_id'].str.replace(r'^1442', '', regex=True)
date_ori.loc[date_ori['ori_temp_id'].str.len() == 12, 'ori_temp_id'] = date_ori['ori_temp_id'].str.replace(r'^1012', '', regex=True)
date_ori.loc[date_ori['ori_temp_id'].str.len() == 10, 'ori_temp_id'] = date_ori['ori_temp_id'].str.replace(r'^47', '', regex=True)

# cambio los años para ajustarme al formato dd/mm/yy
date_ori['Fecha de la pueba'] = date_ori['Fecha de la pueba'].str.replace('/2020', '/20')
date_ori['Fecha de la pueba'] = date_ori['Fecha de la pueba'].str.replace('/2021', '/21')
date_ori['Fecha de la pueba'] = date_ori['Fecha de la pueba'].str.replace('/2022', '/22')
date_ori['Fecha de la pueba'] = date_ori['Fecha de la pueba'].str.replace('/2023', '/23')
date_ori['Fecha de la pueba'] = date_ori['Fecha de la pueba'].str.replace('/2024', '/24')
date_ori['Fecha de la pueba'] = date_ori['Fecha de la pueba'].str.replace('/2025', '/25')
# Veo que campos de mes superan el 12 cuando no deberían e intercambio el día por el mes para ajustarme al formato dd-mm-yy
date_ori[['dd', 'mm', 'yy']] = date_ori['Fecha de la pueba'].str.split('/', expand=True)
date_ori['mmint'] = date_ori['mm'].astype(int)
date_ori.loc[date_ori['mmint'] > 12, 'Fecha de la pueba'] = date_ori['mm'].astype(str) + '-' + date_ori['dd'].astype(str) + '-' + date_ori['yy'].astype(str)

# comprueba que están en el formado adecuado
date_ori['ori_temp_date'] = date_ori['Fecha de la pueba']
date_ori_errors_num = date_ori[['ori_temp_date']].loc[pd.to_datetime(date_ori['ori_temp_date'], format='%d/%m/%y', errors='coerce').isnull()].shape[0]

if date_ori_errors_num > 0:
    print('>>> Fechas formato dd/mm/yy: ')
    print(date_ori[['ori_temp_id', 'ori_temp_date', 'Fecha de la pueba']].loc[pd.to_datetime(date_ori['ori_temp_date'], format='%d/%m/%y', errors='coerce').isnull()])
    # Me quedo con los errores para mandarlos a micro
    date_ori_errors = date_ori[['ori_temp_id', 'ori_temp_date', 'Fecha de la pueba']].loc[pd.to_datetime(date_ori['ori_temp_date'], format='%d/%m/%y', errors='coerce').isnull()]
    date_ori['ori_temp_date'] = pd.to_datetime(date_ori['ori_temp_date'], format='%d/%m/%y', errors='coerce')
else:
    print('>>> Fechas formato OK')
    date_ori['ori_temp_date'] = pd.to_datetime(date_ori['ori_temp_date'], format='%d/%m/%y')

# Comprueba que no hay fechas del futuro
today = datetime.datetime.now()
if date_ori.loc[date_ori['ori_temp_date'] > today].shape[0]:
    print('>>> Fechas futuro: ', date_ori.loc[date_ori['ori_temp_date'] > today].shape[0])
    print(date_ori[['ori_temp_id', 'NUHSA', 'Fecha de la pueba', 'ori_temp_date']].loc[date_ori['ori_temp_date'] > today])
    date_ori_errors = pd.concat([date_ori_errors, date_ori[['ori_temp_id', 'NUHSA', 'Fecha de la pueba', 'ori_temp_date']].loc[date_ori['ori_temp_date'] > today]])
    date_ori['ori_temp_date'] = date_ori['ori_temp_date'].loc[date_ori['ori_temp_date'] <= today]
else:
    print('>>> Fechas futuro OK')
# Comprueba que no hay fechas anteriores al 1 enero de 2020
if date_ori.loc[date_ori['ori_temp_date'] < '2020-01-01'].shape[0]:
    print('>>> Fechas antiguas :')
    print(date_ori[['ori_temp_id', 'NUHSA', 'Fecha de la pueba', 'ori_temp_date']].loc[date_ori['ori_temp_date'] < '2020-01-01'])
    date_ori_errors = pd.concat([date_ori_errors, date_ori[['ori_temp_id', 'NUHSA', 'Fecha de la pueba', 'ori_temp_date']].loc[date_ori['ori_temp_date'] < '2020-01-01']])
    date_ori['ori_temp_date'] = date_ori['ori_temp_date'].loc[date_ori['ori_temp_date'] >= '2020-01-01']
else:
    print('>>> Fechas antiguas OK')
date_ori_errors.to_csv('date_ori_errors.tsv', sep='\t', index=False)

# elimino las fechas duplicadas con mismo id y nuhsa
date_ori.drop(date_ori.loc[date_ori[['ori_temp_date', 'ori_temp_id']].duplicated(keep='first')].index, inplace=True)
date_ori.drop(date_ori.loc[date_ori[['NUHSA', 'ori_temp_id']].duplicated(keep='first')].index, inplace=True)

# elimino los números de laboratorio con diferente NUHSA
ids = set(date_ori.loc[date_ori['ori_temp_id'].duplicated(keep=False)].sort_values('ori_temp_id')['ori_temp_id'].values)
for i in ids:
#     print(i)
    num_dif_nuhsa = len(set(date_ori.loc[date_ori['ori_temp_id'] == i]['NUHSA'].values))
    if num_dif_nuhsa > 1:
#         print(num_dif_nuhsa)
        date_ori.drop(date_ori.loc[date_ori['ori_temp_id'] == i].index, inplace=True)

# mergeo las tablas de samples y microbiología oriental
print(date_ori.columns)
samples = samples.merge(date_ori[['ori_temp_date', 'ori_temp_id']], how='left', left_on='laboratoryID', right_on='ori_temp_id')
samples['fecha_micro_husc'] = samples['ori_temp_date']

# Unifico las fechas procedentes de microbiología oriental y  occidental
samples.loc[(samples['ori_temp_date'].notnull()) & (samples['Fecha_micro'].isnull()), 'Fecha_micro'] = samples['ori_temp_date']

# genero una tabla con las muestras que no tienen fecha de mircrobiologia
samples.loc[samples['Fecha_micro'].isnull(), ['sample', 'laboratoryID', 'NUHSA', 'batch', 'Fecha Inicio Síntomas (dd/mm/aaaa)', 'Fecha_micro']].to_excel('muestras_sin_fecha_micro.xlsx')

# unifico columnas y elimino las sobrantes de las dos tablas
samples.loc[samples['Fecha_micro'].notnull(), 'fecha_nextstrain'] = samples['Fecha_micro']
samples.drop(columns=['ori_temp_date', 'ori_temp_id'], inplace=True)

samples.loc[samples.duplicated(keep=False)].shape

#########################################
Andalucia Oriental (Microbiología HUSC) #
#########################################
>>> Fechas formato dd/mm/yy: 
     ori_temp_id ori_temp_date Fecha de la pueba
253     22217049    01/02/1998        01/02/1998
437     35219490    24/08/1988        24/08/1988
3680    23112077    24/07/1994        24/07/1994
5622    30644492    27/10/2010        27/10/2010
>>> Fechas futuro OK
>>> Fechas antiguas OK
Index(['ori_temp_id', 'NUHSA', 'Fecha de la pueba', 'dd', 'mm', 'yy', 'mmint',
       'ori_temp_date'],
      dtype='object')


(0, 75)

In [67]:
samples.loc[samples['batch'].str.contains('huvr')].shape

(23838, 75)

In [68]:
#############################################################
### Parseo las las fechas y los labID del fichero de HRUM ###
#############################################################
print('#########################################')
print('# Málaga (Microbiología HRUM)           #')
print('#########################################')
errors = []

date_malaga_path = f'{ssp}/metadata/hrum/HRUM_INFORME_latest.csv'
date_malaga = pd.read_csv(date_malaga_path, sep=';', dtype=str)
date_malaga = date_malaga[['Petición', 'Fecha registro']]

# Guardo las muestras sin fecha en un dataframe para reportar errores
errors.append(date_malaga.loc[date_malaga['Fecha registro'].isnull()])

# transformo las fechas a datetime y me quedo con los errore
errors.append(date_malaga.loc[pd.to_datetime(date_malaga['Fecha registro'], format='%m/%d/%Y', errors='coerce').isnull()])
date_malaga['Fecha registro'] = pd.to_datetime(date_malaga['Fecha registro'], format='%m/%d/%Y', errors='coerce')

# Comprueba que no hay fechas del futuro
today = datetime.datetime.now()
if date_malaga.loc[date_malaga['Fecha registro'] > today].shape[0]:
    print('>>> Fechas futuro: ', date_malaga.loc[date_malaga['Fecha registro'] > today].shape[0])
    print(date_malaga.loc[date_malaga['Fecha registro'] > today])
    errors.append(date_malaga.loc[date_malaga['Fecha registro'] > today])
    date_malaga['Fecha registro'] = date_malaga['Fecha registro'].loc[date_malaga['Fecha registro'] <= today]
else:
    print('>>> Fechas futuro OK')
    
# Comprueba que no hay fechas anteriores al 1 enero de 2020
if date_malaga.loc[date_malaga['Fecha registro'] < '2020-01-01'].shape[0]:
    print('>>> Fechas antiguas :')
    print(date_malaga.loc[date_malaga['Fecha registro'] < '2020-01-01'])
    errors.append(date_malaga.loc[date_malaga['Fecha registro'] < '2020-01-01'])
    date_malaga['Fecha registro'] = date_malaga['Fecha registro'].loc[date_malaga['Fecha registro'] >= '2020-01-01']
else:
    print('>>> Fechas antiguas OK')

# mergeo las tablas de samples y microbiología hurm
samples = samples.merge(date_malaga, how='left', left_on='laboratoryID', right_on='Petición')
samples['fecha_micro_hurm'] = samples['Fecha registro']

# Unifico las fechas procedentes de microbiología oriental y  occidental
samples.loc[(samples['Fecha registro'].notnull()) & (samples['Fecha_micro'].isnull()), 'Fecha_micro'] = samples['Fecha registro']

# unifico columnas y elimino las sobrantes de las dos tablas
samples.loc[samples['Fecha_micro'].notnull(), 'fecha_nextstrain'] = samples['Fecha_micro']
samples.drop(columns=['Petición', 'Fecha registro'], inplace=True)


#########################################
# Málaga (Microbiología HRUM)           #
#########################################
>>> Fechas futuro OK
>>> Fechas antiguas OK


In [69]:
samples.loc[samples['batch'].str.contains('huvr')].shape

(23838, 76)

In [70]:
########################################################
# Actualizo las fechas según los datos de Microbiología 
############ Andalucía Montañosa #######################
print('#########################################')
print('Andalucia Montañosa (Microbiología HUVN)')
print('#########################################')
date_monta_path = f'{metadata_path}/huvn/INFORME_ACUMULADO_SECUENCIACION_SARS-COV-2_actualizado_latest.csv'
date_monta = pd.read_csv(date_monta_path, sep=';', dtype=str)
date_monta['date_huvn'] = date_monta['Fecha de toma de muestra']
date_monta.loc[date_monta['date_huvn'].isnull(), 'date_huvn'] = date_monta['Fecha de recepción de la muestra']

# transformo las fechas a datetime y me quedo con los errore
format_errors_num = date_monta.loc[pd.to_datetime(date_monta['date_huvn'], format='%d/%m/%Y', errors='coerce').isnull()].shape[0]
print(f'>>> Errors in date format %d/%m/%Y: {format_errors_num}')
date_monta['date_huvn'] = pd.to_datetime(date_monta['date_huvn'], format='%d/%m/%Y', errors='coerce')

# Comprueba que no hay fechas del futuro
today = datetime.datetime.now()
if date_monta.loc[date_monta['date_huvn'] > today].shape[0]:
    print('>>> Fechas futuro: ', date_monta.loc[date_monta['date_huvn'] > today].shape[0])
    print(date_monta.loc[date_monta['date_huvn'] > today])
    errors.append(date_monta.loc[date_monta['date_huvn'] > today])
    date_monta['date_huvn'] = date_monta['date_huvn'].loc[date_monta['date_huvn'] <= today]
else:
    print('>>> Fechas futuro OK')
    
# Comprueba que no hay fechas anteriores al 1 enero de 2020
if date_monta.loc[date_monta['date_huvn'] < '2020-01-01'].shape[0]:
    print('>>> Fechas antiguas :')
    print(date_monta.loc[date_monta['date_huvn'] < '2020-01-01'])
    errors.append(date_monta.loc[date_monta['date_huvn'] < '2020-01-01'])
    date_monta['date_huvn'] = date_monta['date_huvn'].loc[date_monta['date_huvn'] >= '2020-01-01']
else:
    print('>>> Fechas antiguas OK')

# mergeo las tablas de samples y microbiología huvn
samples = samples.merge(date_monta[['date_huvn', 'Número']], how='left', left_on='laboratoryID', right_on='Número')
samples['fecha_micro_huvn'] = samples['date_huvn']

# Unifico las fechas procedentes de microbiología oriental y  occidental
samples.loc[(samples['date_huvn'].notnull()) & (samples['Fecha_micro'].isnull()), 'Fecha_micro'] = samples['date_huvn']

# Unifico columnas y elimino las sobrantes de las dos tablas
samples.loc[samples['Fecha_micro'].notnull(), 'fecha_nextstrain'] = samples['Fecha_micro']
samples.drop(columns=['Número', 'date_huvn'], inplace=True)


#########################################
Andalucia Montañosa (Microbiología HUVN)
#########################################
>>> Errors in date format %d/%m/%Y: 2
>>> Fechas futuro OK
>>> Fechas antiguas OK


In [71]:
###############################
####### test de fechas ########
date_columns=[
    'sample','NUHSA', 'laboratoryID',
    'Fecha del caso', # Salud Publica
    'Fecha_diag', # Salud Publica,
    'Fecha_micro_huvr',
    'fecha_micro_husc',
    'fecha_micro_hurm',
    'fecha_micro_huvn',
    'Fecha_micro', # Fechas de micro unificadas
    'fecha_nextstrain'
]

test_ids = ['AND31602', 'AND33747', 'AND33736', 'AND33695', 'AND33748', 'AND33684', 'AND31607'] # esta muestra es del virgen de las nieves con fecha de salud publica del 20 y de micro del 22

samples.loc[samples['sample'].isin(test_ids) , date_columns]


,sample,NUHSA,laboratoryID,Fecha del caso,Fecha_diag,Fecha_micro_huvr,fecha_micro_husc,fecha_micro_hurm,fecha_micro_huvn,Fecha_micro,fecha_nextstrain
31536,AND31602,AN0353134056,11306019,NaN,05/04/2020,NaT,NaT,NaT,NaT,NaT,2020-04-05
31541,AND31607,AN0368827545,11434795,NaN,05/04/2020,NaT,NaT,NaT,NaT,NaT,2020-04-05
33569,AND33684,AN0014340944,80966346,NaN,19/05/2022,NaT,NaT,NaT,NaT,NaT,2022-05-19
33580,AND33695,AN0015762804,80966357,NaN,25/01/2022,NaT,NaT,NaT,NaT,NaT,2022-01-25
33621,AND33736,AN0533710569,28257783,NaN,05/11/2020,NaT,NaT,NaT,NaT,NaT,2020-11-05
33632,AND33747,AN0698638558,50548159,NaN,17/10/2020,NaT,NaT,NaT,NaT,NaT,2020-10-17
33633,AND33748,AN0817235408,50548194,NaN,16/02/2022,NaT,NaT,NaT,NaT,NaT,2022-02-16


In [72]:
lou_dates = samples[['sample', 'fecha_nextstrain']].rename(columns={'fecha_nextstrain':'date'})
lou_dates.to_csv('./rest_waves_sample_date.tsv', sep='\t', index=False)

In [73]:
print(samples.shape)
print(mapping.shape)
print(metadata.shape)
print(lineage.shape)

(43393, 77)
(43362, 3)
(44156, 17)
(43885, 16)


In [74]:
print(samples.shape)
print(mapping.shape)
print(metadata.shape)
print(lineage.shape)

(43393, 77)
(43362, 3)
(44156, 17)
(43885, 16)


In [75]:
set(samples['sample'].values) - set(mapping['sample'].values)

{'AND13515',
 'AND13516',
 'AND13517',
 'AND13518',
 'AND13519',
 'AND13520',
 'AND13521',
 'AND13522',
 'AND13523',
 'AND13524',
 'AND13525',
 'AND13526',
 'AND13527',
 'AND13528',
 'AND13529',
 'AND13530',
 'AND13531',
 'AND13532',
 'AND13533',
 'AND13534',
 'AND13535',
 'AND13536',
 'AND13537'}

In [76]:
# Muestra los duplicados
dup = samples.loc[samples['sample'].duplicated(keep=False)]
dup = dup.fillna(0)
dup.drop_duplicates(keep=False, inplace=True)
# dup = dup.loc[dup['laboratoryID'].duplicated(keep=False)]

dup[['sample', 
     'laboratoryID', 
     'NUHSA',
     'selected_for_nextstrain',
     'batch',
     'Sexo',
     'Edad (años)',
     'fecha_nextstrain',
     'Fecha del caso',
     'hospital',
     'hospital de referencia',
     'Municipio',
     'provincia'
    ]].groupby('laboratoryID').agg({
                            'sample':set,
                            'laboratoryID':'count',
                            'NUHSA':set,
                            'selected_for_nextstrain':set,
                            'batch':set,
                            'Sexo':set,
                            'Edad (años)':set,
                            'fecha_nextstrain':set,
                            'Fecha del caso':set,
                            'hospital':set,
                            'hospital de referencia':set,
                            'Municipio':set,
                            'provincia':set
                           }).sort_index()

/tmp/ipykernel_50290/1549082106.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dup = dup.fillna(0)
/tmp/ipykernel_50290/1549082106.py:3: FutureWarning: Downcasting behavior in Series and DataFrame methods 'where', 'mask', and 'clip' is deprecated. In a future version this will not infer object dtypes or cast all-round floats to integers. Instead call result.infer_objects(copy=False) for object inference, or cast round floats explicitly. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dup = dup.fillna(0)


,sample,laboratoryID,NUHSA,selected_for_nextstrain,batch,Sexo,Edad (años),fecha_nextstrain,Fecha del caso,hospital,hospital de referencia,Municipio,provincia
laboratoryID,,,,,,,,,,,,,
1512915,{AND26568},2,{AN0232313886},{no},{AND_232_huvr_290622},{0},{0},"{2023-08-24 00:00:00, 2022-06-16 00:00:00}",{0},{Hospital Universitario Juan Ramón Jiménez},{HUVR},{Aljaraque},{Huelva}


In [77]:
samples.to_csv(f'{ssp}/analysis_MN908947/other/complete_samples_metadata.tsv', sep='\t')

In [78]:
auspice = samples[
    [
    'sample',
    'laboratoryID',
    'NUHSA',    
    'fecha_nextstrain',
    'País del caso',
    'provincia',
    'Municipio',
    'origen',
    'hospital',
    'hospital de referencia',
    'Centro de salud',
    'centro_secuenciacion',
    'lineage',
    'clade',
    'clade_display',
    'nextstrain_genome',
    'selected_for_nextstrain',
    'batch',
    'percentage_reference_genome_in_consensus_ivar',
    'seqPlatform'
    ]
].copy()

auspice.rename(columns={
    'sample':'strain',
    'fecha_nextstrain':'date',
    'País del caso':'country',
    'Municipio':'location',
    'origen':'source',
    'centro_secuenciacion':'laboratorio_secuenciación'
}, inplace=True)


### creo las columnas necesarias para nextstrain
auspice['region'] = 'Europe'
auspice['country'] = 'Spain'
auspice['location'] = auspice['location'].str.strip()

# conversion de nombres de localidades
location_names = f'./data/location_names_dict.tsv'
loc = {}
with open(location_names,'r') as f:
    for line in f: 
        line = line.rstrip()
        words = line.split('\t')
        loc[words[0]] = words[1]
for l in loc.keys():
    auspice.loc[:,'location'] = auspice['location'].replace(l, loc[l])

print(f'auspice: {auspice.shape[0]}')



### me quedo solo con las seleccionadas para nextstrains
auspice = auspice.loc[auspice['selected_for_nextstrain'] == 'yes']
print(f'auspice selected for nextstrain: {auspice.shape[0]}')


### concateno con la primera tanda de relcov
primera = pd.read_csv(f'{base}/other_data/1wave_data/metadata.tsv', sep='\t')
primera['1wave'] = 'yes'
primera['nextstrain_genome'] = np.nan
primera['date'] = pd.to_datetime(primera['date'], format='%Y-%m-%d')
### elimino los clados y linages ya que voy a mergearlos con unos más recientes
primera.drop(columns=['clade', 'lineage'], inplace=True)
### mergeo la primera ola con los clados de nextclade actualizados
primera = primera.merge(nextclade[['seqName', 'clade', 'clade_display']], how='left', left_on='strain', right_on='seqName')
### mergeo la primera ola con los linages actualizados
primera = primera.merge(lineage[['taxon', 'lineage']], how='left', left_on='strain', right_on='taxon')
### eliminio los indices creados con los mergeos de nextclade y lineage
primera.drop(columns=['seqName', 'taxon'], inplace=True)
### concateno con las muestras seleccionadas para auspice
auspice = pd.concat([auspice, primera])
print(f'auspice added first wave: {auspice.shape[0]}')




### me quedo con la que tienen fecha
auspice = auspice.loc[auspice['date'].notnull()]
print(f'auspice with date: {auspice.shape[0]}')



### filtros de fecha por linaje
err_date_lineage_alpha = auspice.loc[(auspice['date'] < '2020-12-01') & (auspice['lineage'] == 'B.1.1.7')]
err_date_lineage_delta = auspice.loc[(auspice['date'] < '2021-05-01') & (auspice['lineage'].str.startswith('AY'))]
err_date_lineage_delta_2 = auspice.loc[(auspice['date'] < '2021-05-01') & (auspice['lineage'] == 'B.1.617.2')]
err_date_lineage_omicron = auspice.loc[(auspice['date'] < '2021-12-01') & (auspice['lineage'].str.startswith('BA'))]
err_date_lineage = pd.concat([err_date_lineage_alpha, err_date_lineage_delta, err_date_lineage_delta_2, err_date_lineage_omicron])
err_date_lineage.to_excel('error_date_lineage.xlsx', index=False)

auspice = auspice.loc[~auspice['strain'].isin(err_date_lineage['strain'])]
print(f'auspice after date-lineage filter: {auspice.shape[0]}')



auspice: 43393
auspice selected for nextstrain: 35803
auspice added first wave: 36820
auspice with date: 35953
auspice after date-lineage filter: 35892


In [79]:
### filtro de muestras extrañas a partir de AND id (son recombinantes fuera de fecha)
# ANDid_todel = [
#     'AND31602',
#     'AND31607',
#     'AND33747',
#     'AND33736',
#     'AND33706',
#     'AND31606',
#     'AND33695',
#     'AND33748',
#     'AND33684'
# ]
# auspice = auspice.loc[~auspice['strain'].isin(ANDid_todel)].copy()

##### elimino de la selección las muestras de Jerez ####
# and_jerez = ['AND32737', 'AND32242'] # No tienen fechas adecuadas parece que se han secuenciado meses despues del diagnostico
# auspice = auspice.loc[~auspice['strain'].isin(and_jerez)]
# print(f'auspice after AND id filter: {auspice.shape[0]}')

### Elimino las muestras del fichero remove_SARSCOV2_samples_outliers.txt
### Están incluidos todos los filtros que se basan en AND_id
andid_todel = []
with open(nextstrain_data / 'remove_SARSCOV2_samples_outliers.txt', 'r') as file:
    for line in file:
        line = line.strip()
        andid_todel.append(line)
auspice = auspice.loc[~auspice['strain'].isin(andid_todel)]
print(f'auspice after AND id filter: {auspice.shape[0]}')

### elimino duplicados
auspice.drop_duplicates('strain', keep='first', inplace=True)
print(f'auspice without strain duplicated: {auspice.shape[0]}')

### Compruebo que no existen fechas del futuro
today = datetime.datetime.now()
auspice = auspice.loc[auspice['date'] < today]
print(f'auspice without future dates: {auspice.shape[0]}')

### string vacía para las muestras que no tienen batch
auspice.loc[auspice['batch'].isnull(), 'batch'] = ''


### hago una copia para guardar lo que necesita Javi para sus gráficas
### samples selected for nextstrain with date
today = datetime.datetime.now().strftime('%d%m%y')
web_path = f'{ssp}/analysis_MN908947/reports/web/'
agg_noloc = auspice[[
                    'strain',
                    'batch',
                    'date',
                    'lineage',
                    'location',
                    'provincia',
                    'hospital',
                    'hospital de referencia'
                ]].copy()
agg_noloc.to_csv(f'{web_path}agregado_web_noloc_{today}.tsv', sep='\t', index=False)
agg_noloc.loc[agg_noloc['batch'].str.contains('husc')].to_csv(f'{web_path}agregado_web_noloc_husc_{today}.tsv', sep='\t', index=False)
agg_noloc.loc[agg_noloc['batch'].str.contains('huvn')].to_csv(f'{web_path}agregado_web_noloc_huvn_{today}.tsv', sep='\t', index=False)
agg_noloc.loc[agg_noloc['batch'].str.contains('huvr')].to_csv(f'{web_path}agregado_web_noloc_huvr_{today}.tsv', sep='\t', index=False)

### me quedo con las que tienen localizacion
auspice = auspice.loc[auspice['location'].notnull()]
print(f'auspice with location: {auspice.shape[0]}')

### elimino las que son SAS-imputed
auspice = auspice.loc[auspice['source'] != 'SAS-imputed']
print(f'auspice without SAS-imputed: {auspice.shape[0]}')


### hago una copia del agregado web antiguo por retro compatibilidad
agg = auspice[[
                'strain',
                'batch',
                'date',
                'lineage',
                'location',
                'provincia',
                'hospital',
                'hospital de referencia'
            ]].copy()

agg.to_csv(f'{web_path}agregado_web_{today}.tsv', sep='\t', index=False)
agg.loc[agg['batch'].str.contains('husc')].to_csv(f'{web_path}agregado_web_husc_{today}.tsv', sep='\t', index=False)
agg.loc[agg['batch'].str.contains('huvn')].to_csv(f'{web_path}agregado_web_huvn_{today}.tsv', sep='\t', index=False)
agg.loc[agg['batch'].str.contains('huvr')].to_csv(f'{web_path}agregado_web_huvr_{today}.tsv', sep='\t', index=False)

### transformo las fechas a strings
auspice['date'] = auspice['date'].dt.strftime('%Y-%m-%d')


auspice after AND id filter: 35606
auspice without strain duplicated: 35604
auspice without future dates: 35604
auspice with location: 32341
auspice without SAS-imputed: 32194


In [80]:
auspice['source'].unique()

array(['SAS', 'GISAID'], dtype=object)

In [81]:
# Comprueba que todas las localizaciones tienen latitud y longitud
lat_long = pd.read_csv('config/lat_long.tsv', sep='\t', header=None)
lat_long = set(lat_long[1].values)
locs = set(auspice['location'].values)
locs - lat_long

{'V�lor'}

In [82]:
###### marco las muestras de los últmos 4 meses 
###### para no filtrarlas con augur filter
months = datetime.date.today() - datetime.timedelta(4*365/12)
months = months.strftime(format="%Y-%m-%d")
auspice.loc[pd.to_datetime(auspice['date']) >= months, 'include'] = 'yes'

In [83]:
# tabla para los datos de las gráficas de la web para javier sin filtros
agregado_nofilter = samples[[
    'sample', 
    'lineage',
    'laboratoryID', 
    'batch', 
    'hospital',
    'hospital de referencia',
    'fecha_nextstrain'
        ]].copy()
agregado_nofilter_husc = agregado_nofilter.loc[agregado_nofilter['batch'].str.contains('husc')].copy()
agregado_nofilter_huvn = agregado_nofilter.loc[agregado_nofilter['batch'].str.contains('huvn')].copy()
agregado_nofilter_huvr = agregado_nofilter.loc[agregado_nofilter['batch'].str.contains('huvr')].copy()
agregado_nofilter.to_csv(f'{ssp}/analysis_MN908947/reports/web/agregado_without_QC_{today}.tsv', 
                         sep='\t', 
                         index=False)
agregado_nofilter_husc.to_csv(f'{ssp}/analysis_MN908947/reports/web/agregado_without_QC_husc_{today}.tsv', 
                         sep='\t', 
                         index=False)
agregado_nofilter_huvn.to_csv(f'{ssp}/analysis_MN908947/reports/web/agregado_without_QC_huvn_{today}.tsv', 
                         sep='\t', 
                         index=False)
agregado_nofilter_huvr.to_csv(f'{ssp}/analysis_MN908947/reports/web/agregado_without_QC_huvr_{today}.tsv', 
                         sep='\t', 
                         index=False)


In [84]:
# genero el multifasta con las secuencias consenso
consensus_seq = {}
key = ""

# obtenemos los path de las secuencias consenso de las muestras que tienen metadatos
for i, path in auspice[['strain', 'nextstrain_genome']].loc[auspice['nextstrain_genome'].notnull()].values:
#     print(i, path)
    if path == 'unknown':
        continue
    path = path.replace('/analysis_MN90894/', '/analysis_MN908947/')
    with open(path, 'r') as file:
        lines = file.read().splitlines()
        for line in lines:
            if line.startswith(r'>'):
                key = i
                consensus_seq[key] = ""
                continue
            consensus_seq[key] += line
            
# obtengo las secuencias de las muestras de la primera ola (propias más GISAID)
wave1_seq_fasta = f'{base}/other_data/1wave_data/andalusia_seq.fasta'
ids_1wave = auspice.loc[auspice['1wave'] == 'yes', 'strain'].values
wave1_seq = fasta_get_by_name(fasta=wave1_seq_fasta, ids=ids_1wave)


# obtengo las secuencias de las muestras de GISAID ahora mismo no se usa
# gisaid = f'./data/raw_gisaid/2020-07-06/'            
# ids_gi = auspice.loc[auspice['source'] == 'GISAID', 'strain'].values
# gi_seq = fasta_get_by_name(fasta=f'{gisaid}sequences.fasta', ids=ids_gi)
##############

seq = dict(consensus_seq, **wave1_seq)
with open(f'./data/sequence.fasta', 'w') as file:
    for k in seq:
        file.write(f'>{k}\n')
        file.write(f'{seq[k]}\n')
        
# Comprueba que todos los id de las secuencias están dentro de los metadatos
print(len(seq.keys()), len(auspice['strain']))
print(set(seq.keys()) - set(auspice['strain']))
print(set(auspice['strain']) - set(seq.keys()))

32194 32194
set()
set()


In [ ]:
auspice.to_csv('./data/auspice_metadata.tsv', sep='\t', index=False)

In [ ]:
len(seq.keys())

In [86]:
# Obtengo las secuencias consenso de todas las muestran hayan o no pasado el control de calidad
consensus_seq = {}
for i, path in samples[['sample', 'nextstrain_genome']].loc[samples['nextstrain_genome'].notnull()].values:
#     print(i, path)
    if path == 'unknown' or path == '-':
        continue
    path = path.replace('/analysis_MN90894/', '/analysis_MN908947/')
    with open(path, 'r') as file:
        lines = file.read().splitlines()
        for line in lines:
            if line.startswith(r'>'):
                key = i
                consensus_seq[key] = ""
                continue
            consensus_seq[key] += line

# obtengo las secuencias de las muestras de la primera ola (propias más GISAID)
wave1_consensus_seqs_path = f'{base}/other_data/1wave_data/1wave_all_consensus_sequences.fasta'
wave1_consensus_seqs = {}
with open(wave1_consensus_seqs_path, 'r') as file:
    for line in file:
        line = line.strip()
        if line.startswith(r'>'):
            key = line[1:]
            wave1_consensus_seqs[key] = ''
            continue
        wave1_consensus_seqs[key] += line
            
seq = dict(consensus_seq, **wave1_consensus_seqs)
with open(f'./data/AND1-100_1wave_consensus_sequences.fasta', 'w') as file:
    for k in seq:
        file.write(f'>{k}\n')
        file.write(f'{seq[k]}\n')
        


In [87]:
len(seq.keys())

43529

In [88]:
len(seq.keys())

43529